# TFS Arctis Testing

In [ ]:
%load_ext autoreload
%autoreload 2

from fibsem import utils, acquire
from fibsem.structures import BeamType
import numpy as np
import matplotlib.pyplot as plt

# replace with path to your configuration file
CONFIGURATION_PATH = "/Users/patrickcleeve/Documents/fibsem/fibsem/fibsem/config/arctis-configuration.yaml"
microscope, settings = utils.setup_session(config_path=CONFIGURATION_PATH)

print(f"Using Compustage: {microscope.stage_is_compustage}, Pre-Tilt: {microscope.system.stage.shuttle_pre_tilt} deg")
print(f"Electron Column Tilt: {microscope.system.electron.column_tilt} deg")
print(f"Ion Column Tilt: {microscope.system.ion.column_tilt} deg")

assert microscope.stage_is_compustage, "This notebook requires the compustage to run"
assert microscope.system.stage.shuttle_pre_tilt == 0, "This notebook requires the pre-tilt to be 0"

## Move Flat to Beam

In [ ]:
# move flat to electron
microscope.move_flat_to_beam(BeamType.ELECTRON)
stage_position = microscope.get_stage_position()
print(f"Stage Position Flat to Electron: {stage_position}")
print(f"Stage Tilt: {np.rad2deg(stage_position.t)} deg")

# acquire reference images flat to electron
eb_image, ib_image = acquire.take_reference_images(microscope, settings.image)
fig, ax = plt.subplots(1, 2, figsize=(10, 5))
ax[0].imshow(eb_image.data, cmap='gray')
ax[1].imshow(ib_image.data, cmap='gray')
plt.show()


In [ ]:

# move flat to ion
microscope.move_flat_to_beam(BeamType.ION)
stage_position = microscope.get_stage_position()
print(f"Stage Position Flat to Ion: {stage_position}")
print(f"Stage Tilt: {np.rad2deg(stage_position.t)} deg")

# acquire reference images flat to electron
eb_image, ib_image = acquire.take_reference_images(microscope, settings.image)
fig, ax = plt.subplots(1, 2, figsize=(10, 5))
ax[0].imshow(eb_image.data, cmap='gray')
ax[0].set_title("Electron Beam Image")
ax[1].imshow(ib_image.data, cmap='gray')
ax[0].set_title("Ion Beam Image")
plt.show()


## Stable Movement

These movements maintain coincidence between the beams when moving the stage. You should first make the beams coincident, and then run this cell. After running, the beams should still be coincidence, but the stage should have moved in y by 20um. 

These movements also correct for the imaging perspective, which is calculated based on the stage tilt, shuttle pre-tilt and column tilt. The other way to think about this movement is 'move by dx, dy in the imaging perspective' which is why it also takes a beam_type. 

This perspective correction will likely be the source of error, as for the arctis we need to offset this perspective correction by -180 to account for imaging the back of the stage. The bulk of this correction is in ThermoMicroscope._y_corrected_stage_movement, and it's a bit messy. 


In [ ]:

# go flat to ion
microscope.move_flat_to_beam(BeamType.ION)

# take reference images 
eb_image, ib_image = acquire.take_reference_images(microscope, settings.image)

# get current stage position
stage_position = microscope.get_stage_position()
 
# stable move, dy=20e-6 in Ion
# should move 20e-6 upwards, and maintain coincidence between electron and ion
microscope.stable_move(dx=0, dy=20e-6, beam_type=BeamType.ION)

# get new stage position
new_stage_position = microscope.get_stage_position()

# acquire new images
new_eb_image, new_ib_image = acquire.take_reference_images(microscope, settings.image)

# check movement in y, and z
print(f"Initial Stage Position: {stage_position}")
print(f"New Stage Position: {new_stage_position}")

# plot with lime crosshair on each centre
fig, ax = plt.subplots(2, 2, figsize=(12, 7))
ax[0, 0].imshow(eb_image.data, cmap='gray')
ax[0, 0].plot(eb_image.data.shape[1]//2, eb_image.data.shape[0]//2, '+', color='lime')
ax[0, 0].set_title("Initial Electron Beam Image")
ax[0, 1].imshow(ib_image.data, cmap='gray')
ax[0, 1].plot(ib_image.data.shape[1]//2, ib_image.data.shape[0]//2, '+', color='lime')
ax[0, 1].set_title("Initial Ion Beam Image")
ax[1, 0].imshow(new_eb_image.data, cmap='gray')
ax[1, 0].plot(new_eb_image.data.shape[1]//2, new_eb_image.data.shape[0]//2, '+', color='lime')
ax[1, 0].set_title("New Electron Beam Image")
ax[1, 1].imshow(new_ib_image.data, cmap='gray')
ax[1, 1].plot(new_ib_image.data.shape[1]//2, new_ib_image.data.shape[0]//2, '+', color='lime')
ax[1, 1].set_title("New Ion Beam Image")
plt.show()


## Vertical Move

This movement is for restoring coincidence (and eucentricity) between the beams. First move flat to the ion beam and then move the stage down vertically so that the coincidence of the beams is slightly off (~20um). 

This movement currently relies on the inbuilt y-z link to move the stage vertically, with a imaging perspective correction. The source of error will be depend on whether the stage z-axis is based off the tilt (as it is in other systems), and if it inverts direction when the stage tilt is negative. That is to say, does moving by z=1um move you up in the same way whether the stage tilt is 0 deg or -180 deg? We use the raw coordinate system so the z-axis is up (towards SEM Column)

In [ ]:
# move the stage purely vertical, to realign coincidence
# electron beam should not move, ion should move 20e-6 upwards

# move stage purely vertical

# go flat to ion
microscope.move_flat_to_beam(BeamType.ION)

# take reference images 
eb_image, ib_image = acquire.take_reference_images(microscope, settings.image)

# get current stage position
stage_position = microscope.get_stage_position()

# move stage purely vertical 
microscope.vertical_move(dy=20e-6)

# get new stage position
new_stage_position = microscope.get_stage_position()

# acquire new images
new_eb_image, new_ib_image = acquire.take_reference_images(microscope, settings.image)

# check movement in y, and z
print(f"Initial Stage Position: {stage_position}")
print(f"New Stage Position: {new_stage_position}")

# plot with lime crosshair on each centre
fig, ax = plt.subplots(2, 2, figsize=(12, 7))
ax[0, 0].imshow(eb_image.data, cmap='gray')
ax[0, 0].plot(eb_image.data.shape[1]//2, eb_image.data.shape[0]//2, '+', color='lime')
ax[0, 0].set_title("Initial Electron Beam Image")
ax[0, 1].imshow(ib_image.data, cmap='gray')
ax[0, 1].plot(ib_image.data.shape[1]//2, ib_image.data.shape[0]//2, '+', color='lime')
ax[0, 1].set_title("Initial Ion Beam Image")
ax[1, 0].imshow(new_eb_image.data, cmap='gray')
ax[1, 0].plot(new_eb_image.data.shape[1]//2, new_eb_image.data.shape[0]//2, '+', color='lime')
ax[1, 0].set_title("New Electron Beam Image")
ax[1, 1].imshow(new_ib_image.data, cmap='gray')
ax[1, 1].plot(new_ib_image.data.shape[1]//2, new_ib_image.data.shape[0]//2, '+', color='lime')
ax[1, 1].set_title("New Ion Beam Image")
plt.show()